# 🧪 SpectraMind V50 — Experiment Tracking, DVC DAG & CI (Notebook 09)

**Purpose.** Close the reproducibility loop by wiring **experiment tracking**, **DVC pipeline/DAG**, and **CI artifacts**:
- Track runs (config → metrics → artifacts) with **MLflow** (or structured JSON fallback).
- Define/validate a **DVC DAG** for calibration → train → predict → diagnostics stages.
- Emit CI-friendly artifacts (JSON summaries, HTML reports) and run **self-test** hooks.

This continues the sequence after 08 (ensembles/MC Dropout/COREL) and focuses on **engineering rigor**.


In [ ]:
# ░░ Pre-flight: env, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"tracking_dvc_ci_{RUN_TS}"
ROOT_OUT = "/mnt/data/tracking_dvc_ci"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
CI_OUT = os.path.join(ARTIFACTS, "ci_artifacts")
MLF_OUT = os.path.join(ARTIFACTS, "mlruns")  # default local store if mlflow used
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, CI_OUT, MLF_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str) -> bool:
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "ci": CI_OUT, "mlruns": MLF_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 09 ===")
print(json.dumps(env, indent=2))


## Experiment Tracking: MLflow (with structured JSON fallback)

We attempt to use **MLflow**; if unavailable, we record runs in `events.jsonl` + `summary.json` so CI can ingest metrics.


In [ ]:
MLFLOW_OK = False
try:
    import mlflow  # type: ignore
    MLFLOW_OK = True
except Exception:
    MLFLOW_OK = False

print("MLflow available:", MLFLOW_OK)
EVENTS = os.path.join(ARTIFACTS, "events.jsonl")
SUMMARY = os.path.join(ARTIFACTS, "summary.json")


## Compose Hydra overrides for a small demo run

We keep it light here; use your ablation tooling for full sweeps. The goal is to demonstrate **tracked runs**.


In [ ]:
import json, os

overrides = {
    "data": "ariel_nominal",
    "model": "v50",
    "training.max_epochs": "3",
    "training.batch_size": "16",
    "training.seed": "2025",
    # Diagnostics toggles for predictable outputs
    "diagnostics.export_json": "true",
}
cfg_file = os.path.join(CFG_OUT, "tracking_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)
print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (DRY-RUN if CLI not present)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        with open(os.path.join(ARTIFACTS, "events.jsonl"), "a") as ev:
            ev.write(json.dumps({"ts": RUN_TS, "event": "dry_run", "cmd": result["cmd"]}) + "\n")
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try: result["stdout"] = open(log_path, "r").read()
    except Exception: pass
    try: result["stderr"] = open(err_path, "r").read()
    except Exception: pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Tracked demo run: `train → predict → diagnose`

- If MLflow is present, we start a run, log params/metrics/artifacts.
- Otherwise, we append to `events.jsonl` and assemble `summary.json` for CI.


In [ ]:
import uuid, time, json, os, glob

run_uid = str(uuid.uuid4())

def log_fallback(record: dict):
    with open(EVENTS, "a") as ev:
        ev.write(json.dumps(record) + "\n")

# Start run
if MLFLOW_OK:
    mlflow.set_tracking_uri("file://" + os.path.abspath(MLF_OUT))
    mlflow.set_experiment("spectramind_v50")
    mlflow.start_run(run_name=f"demo_{run_uid}")
    mlflow.log_params(overrides)
else:
    log_fallback({"event": "start_run", "run_id": run_uid, "params": overrides, "ts": time.time()})

# train
cmd_train = ["spectramind", "train", "--config-name", "config_v50.yaml", "+outputs.root_dir=" + ARTIFACTS]
for k, v in overrides.items():
    cmd_train.append(f"+{k}={v}")
res_tr = run_cli(cmd_train, log_name="01_train")
if MLFLOW_OK:
    mlflow.log_text(res_tr["stdout"][:1000], "logs/train_head.txt")
else:
    log_fallback({"event": "train_done", "rc": res_tr["returncode"], "ts": time.time()})

# predict
pred_dir = os.path.join(ARTIFACTS, "pred")
os.makedirs(pred_dir, exist_ok=True)
cmd_pred = ["spectramind", "predict", "--config-name", "config_v50.yaml", "+load.from_checkpoint=true", "+outputs.root_dir=" + pred_dir]
for k, v in overrides.items():
    cmd_pred.append(f"+{k}={v}")
cmd_pred += ["+inference.save_mu_sigma=true"]
res_pr = run_cli(cmd_pred, log_name="02_predict")
if MLFLOW_OK:
    mlflow.log_text(res_pr["stdout"][:1000], "logs/predict_head.txt")
else:
    log_fallback({"event": "predict_done", "rc": res_pr["returncode"], "ts": time.time()})

# diagnose dashboard
diag_dir = os.path.join(ARTIFACTS, "diagnostics")
os.makedirs(diag_dir, exist_ok=True)
dashboard = os.path.join(diag_dir, "diagnostic_report_v1.html")
cmd_diag = ["spectramind", "diagnose", "dashboard", "--out", dashboard]
res_dg = run_cli(cmd_diag, log_name="03_diagnose")
if MLFLOW_OK:
    if os.path.exists(dashboard):
        mlflow.log_artifact(dashboard, artifact_path="reports")
    mlflow.log_text(res_dg["stdout"][:1000], "logs/diagnose_head.txt")
else:
    log_fallback({"event": "diagnose_done", "rc": res_dg["returncode"], "dashboard": dashboard, "ts": time.time()})

# Finish run
if MLFLOW_OK:
    # Example metric emit (replace with real metrics from diagnostics JSON if available)
    mlflow.log_metric("demo_metric_gll", 0.0)
    mlflow.end_run()
else:
    with open(SUMMARY, "w") as f:
        json.dump({"run_id": run_uid, "metrics": {"demo_metric_gll": 0.0}}, f, indent=2)
print("Tracking complete.")


## DVC DAG: emit stage templates

We write **DVC stage YAML fragments** for `calibrate → train → predict → diagnose` so you can paste or integrate into `dvc.yaml`.


In [ ]:
dvc_frag = os.path.join(ARTIFACTS, "dvc_stages.yaml")
frag = f"""
stages:
  calibrate:
    cmd: spectramind calibrate --config-name config_v50.yaml
    deps:
    - configs
    outs:
    - data/calibrated
  train:
    cmd: spectramind train --config-name config_v50.yaml
    deps:
    - data/calibrated
    - configs
    outs:
    - models/v50_checkpoint
  predict:
    cmd: spectramind predict --config-name config_v50.yaml +load.from_checkpoint=true
    deps:
    - models/v50_checkpoint
    outs:
    - outputs/predictions
  diagnose:
    cmd: spectramind diagnose dashboard --out outputs/diagnostic_report_v1.html
    deps:
    - outputs/predictions
    outs:
    - outputs/diagnostic_report_v1.html
"""
open(dvc_frag, "w").write(frag)
print("Wrote DVC stage template ->", dvc_frag)
print(open(dvc_frag).read())


## CI artifacts: package JSON summaries & HTML dashboard

We copy key files to the `ci_artifacts/` folder so a CI job can always upload them.


In [ ]:
import shutil, os, glob

# Collect likely artifacts if present; tolerate absence in DRY-RUN
maybe = []
for path in [
    os.path.join(ARTIFACTS, "diagnostics", "diagnostic_report_v1.html"),
    os.path.join(ARTIFACTS, "summary.json"),
    os.path.join(ARTIFACTS, "events.jsonl"),
]:
    if os.path.exists(path):
        maybe.append(path)

for pth in maybe:
    shutil.copy2(pth, CI_OUT)

print("CI bundle contains:", os.listdir(CI_OUT))


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))
print("CI artifacts:", os.listdir(os.path.join(ARTIFACTS, "ci_artifacts")))


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Hydra config] --> B[Calibrate]
  B --> C[Train (tracked)]
  C --> D[Predict]
  D --> E[Diagnose → HTML]
  C --> F[Metrics JSON/MLflow]
  E --> G[CI Upload]
  F --> G
```

## Next steps
- Wire **real metrics** from diagnostics JSON into MLflow/summary outputs.
- Paste `dvc_stages.yaml` content into `dvc.yaml` and `dvc repro` to validate the DAG on small data.
- Hook this notebook (or the CLI) into **CI** to auto-run `selftest` + produce a **CI bundle** on every PR.
